CNN Classifier for SAT-6
===
Todo
---
- [ ] Log learning rate, loss, and accuracy to `TensorBoardX` along with standard classification stats (recall, precision, f1, confusion matrix etc.)
- [ ] Implement stochastic gradient descent with warm restarts
- [ ] Implement test time augmentation
- [ ] Implement early stopping ans save best weights
- [ ] Achieve overall accuracy of 98.5%

In [ ]:
import os
import pandas as pd
import numpy as np
import torch as t
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
import scipy.io

In [2]:
data_dir = '/Users/tttk1/Desktop/deepsat-sat6/'

In [3]:
os.listdir(data_dir)

['y_test_sat6.csv',
 'sat-6-full.mat',
 'X_train_sat6.csv',
 'sat6annotations.csv',
 'y_train_sat6.csv',
 'X_test_sat6.csv']

In [4]:
class SatelliteDataset(Dataset):
    def __init__(self, X_csv, y_csv, sample_size=1000):
        self.instances = pd.read_csv(X_csv, nrows=sample_size)
        self.labels = pd.read_csv(y_csv, nrows=sample_size)

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        img = self.instances.iloc[idx].values.reshape(-1, 4, 28, 28).clip(0, 255).astype(np.uint8).squeeze(axis=0)
        label = self._get_labels(self.labels.iloc[idx].values)
        return {'chip': img, 'label': label}

    def _get_labels(self, row_values):
        annotations = {'building': 0, 'barren_land': 1,'trees': 2, 'grassland': 3, 'road': 4, 'water': 5}
        labels = [list(annotations.values())[i] for i, x in enumerate(row_values) if x == 1]
        return labels[0]

In [5]:
training_set = SatelliteDataset(X_csv=data_dir+'X_train_sat6.csv', y_csv=data_dir+'y_train_sat6.csv')
validation_set = SatelliteDataset(X_csv=data_dir+'X_test_sat6.csv', y_csv=data_dir+'y_test_sat6.csv')

In [6]:
tfms = tv.transforms.Compose([
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.RandomVerticalFlip(),
    tv.transforms.RandomRotation(90),
    tv.transforms.ToTensor()
])

In [7]:
train_loader = DataLoader(
    training_set,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    validation_set,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [8]:
class SmallCNN(t.nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = t.nn.Sequential(
            t.nn.Conv2d(
                in_channels=4,
                out_channels=32,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(32),
            t.nn.ReLU(),
            t.nn.Conv2d(
                in_channels=32,
                out_channels=32,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(32),
            t.nn.ReLU(),
        )
        self.conv2 = t.nn.Sequential(
            t.nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(64),
            t.nn.ReLU(),
            t.nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3
            ),
            t.nn.BatchNorm2d(64),
            t.nn.ReLU(),
        )
        self.pool = t.nn.MaxPool2d(2)
        self.fc1 = t.nn.Linear(1024, 512)
        self.bn = t.nn.BatchNorm1d(512)
        self.drop_out = t.nn.Dropout2d(0.2)
        self.fc2 = t.nn.Linear(512, 6)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc1(x)
        x = self.bn(x)
        x = t.nn.functional.relu(x)
        x = self.drop_out(x)
        output = self.fc2(x)
        return output

In [9]:
model = SmallCNN()
optim = t.optim.SGD(lr=0.001, momentum=0.9, params=model.parameters())
crit = t.nn.CrossEntropyLoss()

In [10]:
EPOCHS = 5
for e in range(EPOCHS):
    for s, inst in enumerate(train_loader):
        chip = inst['chip'].float()
        output = model(chip)
        loss = crit(output, inst['label'])
        optim.zero_grad()
        loss.backward()
        optim.step()
        print(f'Epoch {e+1} Step {s+1} Loss {loss.item()}')

Epoch 1 Step 1 Loss 1.8429967164993286
Epoch 1 Step 2 Loss 1.9091185331344604
Epoch 1 Step 3 Loss 1.8341984748840332
Epoch 1 Step 4 Loss 1.7506349086761475
Epoch 1 Step 5 Loss 1.6989041566848755
Epoch 1 Step 6 Loss 1.6970797777175903
Epoch 1 Step 7 Loss 1.6056914329528809
Epoch 1 Step 8 Loss 1.5161044597625732
Epoch 1 Step 9 Loss 1.4498621225357056
Epoch 1 Step 10 Loss 1.371385931968689
Epoch 1 Step 11 Loss 1.3060106039047241
Epoch 1 Step 12 Loss 1.1970301866531372
Epoch 1 Step 13 Loss 1.2007756233215332
Epoch 1 Step 14 Loss 1.1175326108932495
Epoch 1 Step 15 Loss 1.031986117362976
Epoch 1 Step 16 Loss 1.0480080842971802
Epoch 2 Step 1 Loss 0.933856725692749
Epoch 2 Step 2 Loss 0.9019320011138916
Epoch 2 Step 3 Loss 0.8606889247894287
Epoch 2 Step 4 Loss 0.754473090171814
Epoch 2 Step 5 Loss 0.6591441631317139
Epoch 2 Step 6 Loss 0.8108971118927002
Epoch 2 Step 7 Loss 0.5695822238922119
Epoch 2 Step 8 Loss 0.7120192646980286
Epoch 2 Step 9 Loss 0.6076651811599731
Epoch 2 Step 10 Loss 0

In [11]:
model.eval()
with t.no_grad():
    total = 0
    correct = 0
    for s, inst in enumerate(val_loader):
        output = model(inst['chip'].float())
        _, predicted = t.max(output.data, 1)
        total += inst['label'].size(0)
        correct += (predicted == inst['label']).sum().item()
    print(f'Accuracy {(100 * correct / total)}')

Accuracy 92.8
